Plot how the currents change with time
====
Might be interesting

In [ ]:
import pathlib

data_path = pathlib.Path(
    "~/geog_rdsf/data/projects/dtop/cmip5/historical/cmip5_historical_mdts_yr5.dat"
).expanduser()
mdata_path = data_path.with_name(data_path.stem + "_meta.txt")

assert mdata_path.exists()
assert data_path.exists()

model = "NorESM1-M"
name = "r2i1p1"
year = 1861

Animate how the currents change by year...
----

In [ ]:
from current_denoising.generation import ioutils

data = ioutils.read_clean_currents(
    data_path, mdata_path, year=year, model=model, name=name
)

In [ ]:
from current_denoising.plotting import maps

maps.imshow(data)

In [ ]:
import numpy as np
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

# Years from 1851 to 2001
# CMIP6 also has 2006, but CMIP5 doesn't
years = np.arange(1851, 2006, 5)

fig, ax = plt.subplots(figsize=(10, 5))

vmin, vmax = np.quantile(data[np.isfinite(data)], [0.01, 0.99])
im = ax.imshow(data, vmin=vmin, vmax=vmax, cmap="turbo")

fig.colorbar(im)


def update(year):
    data = ioutils.read_clean_currents(
        data_path,
        mdata_path,
        year=year,
        model=model,
        name=name,
    )

    im.set_array(data)
    ax.set_title(f"Year: {year} Model: {model}")
    return [im]


ani = FuncAnimation(fig, update, frames=tqdm(years), blit=True)

In [ ]:
ani.save(
    "changing_currents.mp4",
    writer="ffmpeg",
    fps=6,
    bitrate=2000,
    extra_args=["-vcodec", "libx264", "-pix_fmt", "yuv420p", "-preset", "fast"],
)

How to get rid of the infinity
====
There's an infinity at the equator, caused by the coriolis term blowing up.

The "normal" way to get rid of this would be to just set the coriolis parameter at latitude=0 to the value of its neighbouring grid points

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from current_denoising.generation import ioutils

mdt = ioutils.read_clean_mdt(data_path, mdata_path, year=year, model=model, name=name)

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(mdt, cmap="viridis")
plt.colorbar(label="Mean Dynamic Topography (m)")

In [ ]:
import numpy as np
from current_denoising.utils import util

lats, _ = util.lat_long_grid(mdt.shape)
coriolis = ioutils._coriolis_parameter(lats)
clipped = ioutils.clipped_coriolis_param(lats, 2.5)

fig, axis = plt.subplots(figsize=(8, 4))

axis.plot(lats, 1 / coriolis, "--", label="Raw")
axis.plot(lats, 1 / clipped, label="clipped")
axis.legend()

axis.set_ylabel("1/Coriolis parameter (s)")
axis.set_xlabel("Latitude (degrees)")

fig.suptitle("Clipped vs raw coriolis parameter")

In [ ]:
from current_denoising.plotting import maps

for clip in [0.1, 2.5, 12.5]:
    maps.imshow(ioutils.current_speed_from_mdt(mdt, clip_at=clip)).suptitle(
        f"Clipped at {clip} $\degree$"
    )